In [1]:
import cv2
from cv2 import aruco
from datetime import datetime
from loguru import logger
from pathlib import Path
from typing import Optional, cast, Final
import awkward as ak
from cv2.typing import MatLike
import numpy as np
from matplotlib import pyplot as plt
import awkward as ak
from awkward import Record as AwkwardRecord, Array as AwkwardArray

In [2]:
NDArray = np.ndarray
OBJECT_POINTS_PARQUET = Path("output") / "object_points.parquet"
DICTIONARY: Final[int] = aruco.DICT_4X4_50
# 400mm
MARKER_LENGTH: Final[float] = 0.4

A_CALIBRATION_PARQUET = Path("output") / "a-ae_08.parquet"
B_CALIBRATION_PARQUET = Path("output") / "b-af_03.parquet"

In [3]:
aruco_dict = aruco.getPredefinedDictionary(DICTIONARY)
def read_camera_calibration(path: Path) -> tuple[MatLike, MatLike]:
    cal = ak.from_parquet(path)[0]
    camera_matrix = cast(MatLike, ak.to_numpy(cal["camera_matrix"]))
    distortion_coefficients = cast(MatLike, ak.to_numpy(cal["distortion_coefficients"]))
    return camera_matrix, distortion_coefficients

a_mtx, a_dist = read_camera_calibration(A_CALIBRATION_PARQUET)
b_camera_matrix, b_distortion_coefficients = read_camera_calibration(B_CALIBRATION_PARQUET)
ops = ak.from_parquet(OBJECT_POINTS_PARQUET)
detector = aruco.ArucoDetector(
    dictionary=aruco_dict, detectorParams=aruco.DetectorParameters()
)

total_ids = cast(NDArray, ak.to_numpy(ops["ids"])).flatten()
total_corners = cast(NDArray, ak.to_numpy(ops["corners"])).reshape(-1, 4, 3)
ops_map: dict[int, NDArray] = dict(zip(total_ids, total_corners))
display("ops_map", ops_map)

'ops_map'

{16: array([[0.152, 0.025, 0.   ],
        [0.249, 0.025, 0.   ],
        [0.249, 0.122, 0.   ],
        [0.152, 0.122, 0.   ]]),
 17: array([[0.025, 0.152, 0.   ],
        [0.122, 0.152, 0.   ],
        [0.122, 0.249, 0.   ],
        [0.025, 0.249, 0.   ]]),
 18: array([[0.27900001, 0.152     , 0.        ],
        [0.37599999, 0.152     , 0.        ],
        [0.37599999, 0.249     , 0.        ],
        [0.27900001, 0.249     , 0.        ]]),
 19: array([[0.152     , 0.27900001, 0.        ],
        [0.249     , 0.27900001, 0.        ],
        [0.249     , 0.37599999, 0.        ],
        [0.152     , 0.37599999, 0.        ]]),
 20: array([[1.51999995e-01, 1.70838222e-17, 3.86000000e-01],
        [2.48999998e-01, 2.89628965e-17, 3.86000000e-01],
        [2.48999998e-01, 2.30233595e-17, 2.88999999e-01],
        [1.51999995e-01, 1.11442852e-17, 2.88999999e-01]]),
 21: array([[ 2.50000004e-02, -6.24569833e-18,  2.59000005e-01],
        [ 1.22000001e-01,  5.63337574e-18,  2.59000005e-0

In [4]:
def process(
    frame: MatLike,
    cam_mtx: MatLike,
    dist_coeffs: MatLike,
    target: Optional[MatLike] = None,
) -> tuple[MatLike, Optional[MatLike], Optional[MatLike]]:
    if target is None:
        target = frame.copy()
    grey = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    # pylint: disable-next=unpacking-non-sequence
    markers, ids, rejected = detector.detectMarkers(grey)
    # `markers` is [N, 1, 4, 2]
    # `ids` is [N, 1]
    ret_rvec: Optional[MatLike] = None
    ret_tvec: Optional[MatLike] = None
    if ids is not None:
        markers = np.reshape(markers, (-1, 4, 2))
        ids = np.reshape(ids, (-1, 1))
        # logger.info("markers={}, ids={}", np.array(markers).shape, np.array(ids).shape)
        ips_map: dict[int, NDArray] = {}
        for cs, id in zip(markers, ids):
            id = int(id)
            cs = cast(NDArray, cs)
            ips_map[id] = cs
            center = np.mean(cs, axis=0).astype(int)
            GREY = (128, 128, 128)
            # logger.info("id={}, center={}", id, center)
            cv2.circle(target, tuple(center), 5, GREY, -1)
            cv2.putText(
                target,
                str(id),
                tuple(center),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                GREY,
                2,
            )
            # BGR
            RED = (0, 0, 255)
            GREEN = (0, 255, 0)
            BLUE = (255, 0, 0)
            YELLOW = (0, 255, 255)
            color_map = [RED, GREEN, BLUE, YELLOW]
            for color, corners in zip(color_map, cs):
                corners = corners.astype(int)
                target = cv2.circle(target, corners, 5, color, -1)
        # https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga50620f0e26e02caa2e9adc07b5fbf24e
        ops: NDArray = np.empty((0, 3), dtype=np.float32)
        ips: NDArray = np.empty((0, 2), dtype=np.float32)
        for id, ip in ips_map.items():
            try:
                op = ops_map[id]
                assert ip.shape == (4, 2), f"corners.shape={ip.shape}"
                assert op.shape == (4, 3), f"op.shape={op.shape}"
                ops = np.concatenate((ops, op), axis=0)
                ips = np.concatenate((ips, ip), axis=0)
            except KeyError:
                logger.warning("No object points for id={}", id)
                continue
        assert len(ops) == len(ips), f"len(ops)={len(ops)} != len(ips)={len(ips)}"
        if len(ops) > 0:
            # https://docs.opencv.org/4.x/d5/d1f/calib3d_solvePnP.html
            # https://docs.opencv.org/4.x/d5/d1f/calib3d_solvePnP.html#calib3d_solvePnP_flags
            ret, rvec, tvec = cv2.solvePnP(
                objectPoints=ops,
                imagePoints=ips,
                cameraMatrix=cam_mtx,
                distCoeffs=dist_coeffs,
                flags=cv2.SOLVEPNP_SQPNP,
            )
            # ret, rvec, tvec, inliners = cv2.solvePnPRansac(
            #     objectPoints=ops,
            #     imagePoints=ips,
            #     cameraMatrix=camera_matrix,
            #     distCoeffs=distortion_coefficients,
            #     flags=cv2.SOLVEPNP_SQPNP,
            # )
            if ret:
                cv2.drawFrameAxes(
                    target,
                    cam_mtx,
                    dist_coeffs,
                    rvec,
                    tvec,
                    MARKER_LENGTH,
                )
                ret_rvec = rvec
                ret_tvec = tvec
    return target, ret_rvec, ret_tvec

In [5]:
A_IMG = Path("dumped/batch_two/op/video-20241219-142434-op-a.png")
B_IMG = Path("dumped/batch_two/op/video-20241219-142439-op-b.png")
a_img = cv2.imread(str(A_IMG))
b_img = cv2.imread(str(B_IMG))

In [6]:
a_result_img, a_rvec, a_tvec = process(a_img, a_mtx, a_dist)
# plt.imshow(cv2.cvtColor(a_result_img, cv2.COLOR_BGR2RGB))

/var/folders/cj/0zmvpygn7m72m42lh6x_hcgw0000gn/T/ipykernel_28395/542219436.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  id = int(id)


In [7]:
b_result_img, b_rvec, b_tvec = process(b_img, b_camera_matrix, b_distortion_coefficients)
# plt.imshow(cv2.cvtColor(b_result_img, cv2.COLOR_BGR2RGB))

/var/folders/cj/0zmvpygn7m72m42lh6x_hcgw0000gn/T/ipykernel_28395/542219436.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  id = int(id)


In [8]:
params = AwkwardArray(
    [
        {
            "name": "a",
            "rvec": a_rvec,
            "tvec": a_tvec,
            "camera_matrix": a_mtx,
            "distortion_coefficients": a_dist,
        },
        {
            "name": "b",
            "rvec": b_rvec,
            "tvec": b_tvec,
            "camera_matrix": b_camera_matrix,
            "distortion_coefficients": b_distortion_coefficients,
        },
    ]
)
display("params", params)
ak.to_parquet(params, Path("output") / "params.parquet")

'params'

<Array [{name: 'a', rvec: [...], ...}, ...] type='2 * {name: string, rvec: ...'>

  created_by: parquet-cpp-arrow version 14.0.1
  num_columns: 5
  num_rows: 2
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0

In [9]:
cv2.imwrite("output/a_result_img.png", a_result_img)
cv2.imwrite("output/b_result_img.png", b_result_img)

True